In [1]:
base_experiment_path = 'data\\hp_configurations'

# Hyperparameter Optimization

## Component to optimize

In [2]:
to_optimize_configuration_dict = None
to_optimize_configuration_path = None

### Use dict definition for component to optimize

In [3]:
from automl.base_configurations.environment.cart_pole import dqn_sb3 as base_rl_configuration
#from automl.base_configurations.environment.cart_pole import ppo_sb3 as base_rl_configuration

to_optimize_configuration_dict = base_rl_configuration.config_dict()

## Generate HP Optimization component

In [4]:
hp_optimization_pipeline_path = None

### Generate Hp Suggestions

In [5]:
from automl.hp_opt.hyperparameter_suggestions_generator import generate_hyperparameter_suggestion_list_for_config_dict
from automl.hp_opt.hyperparameter_suggestion import HyperparameterSuggestion


generated_hp_suggestions_list : list[HyperparameterSuggestion] = generate_hyperparameter_suggestion_list_for_config_dict(to_optimize_configuration_dict)

c:\Users\rgoncalo\anaconda3\envs\AIPython\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
import json

from automl.utils.json_utils.json_component_utils import ComponentValuesElementsEncoder


encoded_generated_hp_suggestions_list = json.dumps(generated_hp_suggestions_list, cls=ComponentValuesElementsEncoder, source_component=None, indent=4)

In [7]:
print(encoded_generated_hp_suggestions_list)

[
    {
        "__type__": "<class 'automl.hp_opt.hyperparameter_suggestion.HyperparameterSuggestion'>",
        "object": {
            "name": "target_update_rate",
            "localizations": [
                [
                    "input",
                    "rl_trainer",
                    1,
                    "agents_trainers_input",
                    "learner",
                    "__get_input_value__",
                    "target_update_rate"
                ]
            ],
            "suggestion": [
                "float",
                {
                    "low": 0.5,
                    "high": 1.0
                }
            ]
        }
    },
    {
        "__type__": "<class 'automl.hp_opt.hyperparameter_suggestion.HyperparameterSuggestion'>",
        "object": {
            "name": "learning_rate",
            "localizations": [
                [
                    "input",
                    "rl_trainer",
                    1,
                    "age

### Hp Optimization pipeline input

In [8]:
hp_optimization_input = {
    "n_trials" : 200,
    "steps" : 2,
    "base_directory" :  base_experiment_path,
    "hyperparameters_range_list" : generated_hp_suggestions_list
    }

### Evaluator

In [9]:
from automl.rl.evaluators.rl_std_avg_evaluator import LastValuesAvgStdEvaluator
from automl.rl.evaluators.rl_evaluator_player import EvaluatorWithPlayer

evaluator = (EvaluatorWithPlayer, {"base_evaluator" : (LastValuesAvgStdEvaluator, {"value_to_use" : "episode_steps"})})

In [10]:
pruner = None
#pruner = optuna.pruners.PercentilePruner(percentile=25.0)

if pruner != None:
    hp_optimization_input["pruner"] = pruner

### To optimize config

In [11]:
if to_optimize_configuration_dict != None:
    hp_optimization_input["configuration_dict"] = to_optimize_configuration_dict
    
    
if to_optimize_configuration_path != None: 
    hp_optimization_input["base_component_configuration_path"] = to_optimize_configuration_path

### Instantiate hp configuration

In [12]:
from automl.hp_opt.hp_optimization_pipeline import HyperparameterOptimizationPipeline

if hp_optimization_pipeline_path != None: # load the configuration in the path
    
    from automl.utils.json_utils.json_component_utils import gen_component_from_path
    hp_optimization_pipeline = gen_component_from_path(hp_optimization_pipeline_path)

else: # if not, generate empty
    hp_optimization_pipeline = HyperparameterOptimizationPipeline()


hp_optimization_pipeline.pass_input(hp_optimization_input)


In [13]:
hp_optimization_pipeline.setup_files() # partially initialize the hp optimization pipeline with the current input, it can be manually changed before it is used

Setting up files with input: 
{'n_trials': 200, 'steps': 2, 'base_directory': 'data\\hp_configurations', 'hyperparameters_range_list': [<automl.hp_opt.hyperparameter_suggestion.HyperparameterSuggestion object at 0x000002303EC84050>, <automl.hp_opt.hyperparameter_suggestion.HyperparameterSuggestion object at 0x000002303EC21810>], 'configuration_dict': {'__type__': <class 'automl.rl.rl_pipeline.RLPipelineComponent'>, 'name': 'RLPipelineComponent', 'input': {'device': 'cuda', 'environment': (<class 'automl.rl.environment.gymnasium.aec_gymnasium_env.AECGymnasiumEnvironmentWrapper'>, {'environment': 'CartPole-v1'}), 'agents_input': {'policy': (<class 'automl.rl.policy.qpolicy.QPolicy'>, {'model': (<class 'automl.ml.models.neural_model.FullyConnectedModelSchema'>, {'hidden_layers': 2, 'hidden_size': 256})})}, 'rl_trainer': (<class 'automl.rl.trainers.rl_trainer_component.RLTrainerComponent'>, {'limit_total_steps': 50000.0, 'default_trainer_class': <class 'automl.rl.trainers.agent_trainer_com

In [14]:
experiment_directory = hp_optimization_pipeline.get_artifact_directory()

In [15]:
hp_optimization_pipeline.save_configuration() # save the configuration as is to, even if the process is interrupted, we can see the added values

After this, we can make manual changes to the generated json files:

 - to_optimize_configuration.json, which is the base in which we want to optimize the hyperparameters of
 - configuration.json, the definition of the actual hyperparameter optimization experiment 
